In [21]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

# from manipulation.scenarios import SetColor

In [22]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [23]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

grasp1 = parser.AddModels(file_name="../my_sdfs/wsg.sdf")[0]
grasp2 = parser.AddModels(file_name="../my_sdfs/wsg.sdf")[0]

brick = parser.AddModels(file_name="../my_sdfs/cube.sdf")[0]
plant.Finalize()

model_list = [grasp1, grasp2]

In [24]:
B_O = plant.GetBodyByName("base_link", brick)
B_Ggrasp1 = plant.GetBodyByName("body", grasp1)
B_Ggrasp2 = plant.GetBodyByName("body", grasp2)

bodies_list = [B_O, B_Ggrasp1, B_Ggrasp2]

In [25]:

for body in bodies_list:
    if body == B_O:
        AddFrameTriadIllustration(
            plant=plant,
            scene_graph=scene_graph,
            body=body,
            length=0.09
        )
        continue
    AddFrameTriadIllustration(
        plant=plant,
        scene_graph=scene_graph,
        body=body,
        length=0.09
        # opacity=0.2
    )

In [26]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [27]:

# Get the current object, O, pose
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

position_constraints = [[0,0.095,-0.02],[0,0.1,0.02]]
R_GgraspO = {}
X_GgraspO = {}
X_OGgrasp = {}
X_WGgrasp = {}

for i in range(len(position_constraints)):
    R_GgraspO[i] = RotationMatrix.MakeXRotation(np.pi/2).multiply(RotationMatrix.MakeZRotation(np.pi / 2))
    X_GgraspO[i] = RigidTransform(R_GgraspO[i], position_constraints[i])
    X_OGgrasp[i] = X_GgraspO[i].inverse()
    X_WGgrasp[i] = X_WO.multiply(X_OGgrasp[i])


for i in range(len(position_constraints)):
    plant.SetFreeBodyPose(plant_context, bodies_list[i+1], X_WGgrasp[i])
    plant.GetJointByName("left_finger_sliding_joint", model_list[i]).set_translation(
        plant_context, -0.025
    )
    plant.GetJointByName("right_finger_sliding_joint", model_list[i]).set_translation(
        plant_context, 0.025
    )


In [28]:
diagram.ForcedPublish(context)